# TME 1

In [1]:
import numpy as np
#import pandas as pd
#dataset=pd.read_csv("BLOSUM62.txt")

## Part 1 :  Needleman & Wunsch

In [2]:
# Initialisation de l'alphabet
alphabet = ['A','T','G','C']

#Initialisation des valeurs de match, mismatch et gap
match = 1
mismatch = -2
gap = -1

#s1 et s2 sont les deux séquences utilisés pour notre test
s1 = "CATGAC"
s2 = "TCTGAAC"


In [3]:
def mat_dist(alphabet,match,mismatch):
    
    #Calcul de la longueur de l'alphabet
    l_al = len(alphabet)
    
    #Création de la matrice de distance
    mat = np.zeros((l_al,l_al))
    
    #Parcours sur deux dimensions
    for i in range(l_al):
        for j in range(l_al):
            #Attribution des valeurs de match et mismatch
            if(i==j):
                mat[i][j] = match
            else:
                mat[i][j]= mismatch
    return mat

In [4]:
#Calcul de matrice de distance pour atgc
mat = mat_dist(alphabet,match,mismatch)
print(mat)

[[ 1. -2. -2. -2.]
 [-2.  1. -2. -2.]
 [-2. -2.  1. -2.]
 [-2. -2. -2.  1.]]


In [5]:

# Calcul de matrice de score sans le traceback
def matscorewotb(s1,s2):
    
    #Initialisation de la matrice de score
    ms = np.zeros((len(s1)+1,len(s2)+1))
    
    #Récupération des longueurs pour l'itération
    x,y = ms.shape
    for i in range(1,y):
        ms[0][i] =i*gap
    for j in range(1,x):
        ms[j][0] =j*gap
    
    for i in range(1,x):
        for j in range(1,y):
            sig = 0
            if s1[i-1]==s2[j-1]:
                sig = match
            else :
                sig = mismatch
            
            v_diag = ms[i-1,j-1]+sig
            v_up = ms[i-1,j]+gap
            v_left = ms[i,j-1]+gap
            
            ms[i,j] = max([v_diag,v_up,v_left])
    return ms

print(matscorewotb(s1,s2))


[[ 0. -1. -2. -3. -4. -5. -6. -7.]
 [-1. -2.  0. -1. -2. -3. -4. -5.]
 [-2. -3. -1. -2. -3. -1. -2. -3.]
 [-3. -1. -2.  0. -1. -2. -3. -4.]
 [-4. -2. -3. -1.  1.  0. -1. -2.]
 [-5. -3. -4. -2.  0.  2.  1.  0.]
 [-6. -4. -2. -3. -1.  1.  0.  2.]]


In [6]:
# Calcul de matrice de score avec le traceback
def nw(s1,s2):
    
    #Initialisation de la matrice de score
    ms = np.zeros((len(s2)+1,len(s1)+1))
    Dir=[]
    #Récupération des longueurs pour l'itération
    x,y = ms.shape
    for i in range(len(s2)+1):
        L=[]
        for j in range(len(s1)+1):
            L.append([])
        Dir.append(L)
    for i in range(1,x):
        ms[i][0] =-i
        Dir[i][0]=[1]
    for j in range(1,y):
        ms[0][j] =-j
        Dir[0][j]=[2]
    for i in range(1,x):
        for j in range(1,y):
            sig = 0
            if s2[i-1]==s1[j-1]:
                sig = match
            else :
                sig = mismatch
            #Traceback
            S=np.array([ms[i-1,j-1]+sig, ms[i-1,j]+gap, ms[i,j-1]+gap])
            ms[i,j]=max(S)
            index=list(np.where(S==max(S))[0])
            Dir[i][j]=index
    #En diagonale on remonte de la longueur de la séquence en ligne
    #En haut on remonte de diag -1
    #A gauche c'est évident
    sc1 = ""
    sc2 = "" 
    tbval = 0
    i,j=len(Dir)-1,len(Dir[i])-1
    while Dir[i][j]!=[]:
        if 0 in Dir[i][j]:
            i-=1
            j-=1                                 #on va vers cette cellule quand on a une diagonale (dans notre Traceback du moins)
            sc1+=s1[j]
            sc2+=s2[i]
        elif 1 in Dir[i][j]:
            sc1+='_'
            i-=1                                 #on va vers cette cellule quand on monte
            sc2+=s2[i]
        else:
            sc2+='_'
            j-=1                                 #évident
            sc1+=s1[j]                  
    sc1=sc1[::-1]
    sc2=sc2[::-1]
    for c1 in range(len(sc1)):
            if sc1[c1] == sc2[c1]:
                tbval += match
            else :
                if (sc1[c1]=='_' or sc2[c1] == '_'):
                    tbval += gap
                else:
                    tbval +=mismatch
    return sc1,sc2,tbval,Dir


sc1,sc2,tbval,Dir = nw(s1,s2)
print(sc1)
print(sc2)
print(tbval)

_CATG_AC
TC_TGAAC
2


## Part 2 : Petit exemple